In [1]:
%reset -f
import numpy as np
import cupy as cu
import csv
import time
import pandas as pd

# Inputs and constants

In [2]:
#values taken from D:\UIUC sem 3\Research\inital_conds_5-25-23.txt
m1=1
G=0.01720209895**2
dim=2 #no. dimensions- 2D or 3D
b=3 #no of bodies in each case
#log(m2/(m1+m2)),log(m3/(m1+m2)),x3,theta,eccentricty of the binary system
x=np.array([-0.30103,-1,-1.30103,-2])                       #log(m2/(m1+m2))
y=np.array([-2,-3,-4,-5,-6,-7])                             #log(m3/(m1+m2))
z=np.arange(1.5,8.5,0.5)                                    #distance of the 3rd planet from the barycenter of m1, m2
m_all=np.arange(0,2*np.pi,np.pi/4)                          #mean anomaly of the 3rd planet
m=m_all
e=np.arange(0,1,0.1)                                        #eccentricity of the binary stars
f=(X,Y,Z,W,E)=np.meshgrid(x,y,z,m,e)
print(np.shape(f))
c=np.array(np.shape(f))
n=int(np.product(c[1:]))

X=np.reshape(X,(n,1))
Y=np.reshape(Y,(n,1))
Z=np.reshape(Z,(n,1))
W=np.reshape(W,(n,1))
E=np.reshape(E,(n,1))
Mean=W
#solving of eccentric anomaly given mean anomaly
Ee=np.reshape(E,n)
Mm=np.reshape(W,n)

#stacking all cases to create a dataset of shape n*4
Data = np.hstack((X,Y,Z,W,E))
#print(np.shape(Data))

Data[:,0:2]=1/(np.exp(-Data[:,0:2])-1)     

n #total number of cases

(5, 6, 4, 14, 8, 10)


26880

# Initial conditions

In [3]:
import spiceypy as sp

M=np.transpose(np.insert(Data[0:n,0:2],0,(np.ones(n)),axis=1))
Zz=np.reshape(Z[0:n],n)
T=2*np.pi*np.sqrt(1/(G*(M[0]+M[1])))
R=np.zeros((3,1,2))
V=np.zeros((3,1,2))
%timeit 
for i in range(n):
    
    (rx,ry,rz,vx,vy,vz)=sp.conics((1*(1-Ee[i]),Ee[i],0,0,0,0,0,G*(M[0,i]+M[1,i])),T[i]/2)
    r=np.array([rx,ry])
    v=np.array([vx,vy])
    
    r1=M[1,i]*r/(M[0,i]+M[1,i])
    r2=-M[0,i]*r/(M[0,i]+M[1,i])
    v1=M[1,i]*v/(M[0,i]+M[1,i])
    v2=-M[0,i]*v/(M[0,i]+M[1,i])
    
    r3=[Data[i,2]*np.cos(Data[i,3]),Data[i,2]*np.sin(Data[i,3])]#change to mean anomaly
    r3=np.array(r3)
    r3_norm=np.linalg.norm(r3)
    
    v=np.sqrt(G*(1+Data[i,0])/r3_norm)
    v3=[-v*np.sin(Data[i,3]),v*np.cos(Data[i,3])]
    v3=np.array(v3)
    
    rcom=(M[0,i]*r1+M[1,i]*r2+M[2,i]*r3)/(M[0,i]+M[1,i]+M[2,i])
    vcom=(M[0,i]*v1+M[1,i]*v2+M[2,i]*v3)/(M[0,i]+M[1,i]+M[2,i])
    
    Rr=np.array([r1,r2,r3])
    Rr=Rr-rcom
    Rr=np.reshape([Rr],(3,1,2))
    R=np.append(R,Rr,axis=1)
    
    Vv=np.array([v1,v2,v3])
    Vv=Vv-vcom
    Vv=np.reshape([Vv],(3,1,2))
    V=np.append(V,Vv,axis=1)
    
R=R[:,1:]   
V=V[:,1:]

#total time of run
T_all1=2*np.pi*np.sqrt(Zz[:]**3/(G*(M[0,:]+M[1,:]+M[2,:])))

N=np.reshape(M,(b,1,n))

R1=np.reshape(R,(1,b,n,2))
V1=np.reshape(V,(1,b,n,2))

R=cu.array(R)
V=cu.array(V)
M=cu.array(M)
N=cu.array(N)
T_all=cu.array(T_all1)


# Initalizing energies

In [4]:
#relative positions and distances:
diff=cu.zeros((b,b,n,dim))
rho=cu.zeros((b,b,n))
for i in range(b):
    diff[:,i]=cu.array([(R[i]-R[j]) for j in range(b)])
    rho[i]=cu.array([((R[i]-R[j])*(R[i]-R[j])).sum(axis=1)**0.5 for j in range(b)])
    
#Kinetic energy:
T=0.5*M*((V*V).sum(axis=2))
T_sum=T.sum(axis=0)
#print(T_sum)

#Potential energy:
U=G*M/(rho)*N
U[cu.isinf(U)]=0
U_sum=cu.sum(cu.sum(U,axis=0),axis=0)/2

#Binding energy: constant throughout
B=U_sum-T_sum

#pseudo-timestep:
t_orbit=2*np.pi/np.sqrt(G*(1+M[1]))
H_n=U_sum*t_orbit/35 #35 steps per inner binary orbit

#Initializing time at t=0
t=0
t1=t2=t3=0
tim1=tim2=tim3=[]


# LHLA method

In [131]:
def LHLA2(r,v)  :  
    global T_sum,t1,t2,t3,U_sum,n,dt1,h_n
    #X(h/2)
    #h=h_n/2
    
    dt1=0.5*h_n/(B+T_sum)
    dt1=cu.array(np.reshape(dt1,(n,1)))
    t1=t1+dt1    
    r=r+dt1*v                               #position update
    
    for i in range(b):
        for j in range(b):
            diff[i,j]=r[j]-r[i]
            rho[i,j]=(diff[i,j]*diff[i,j]).sum(axis=1)**0.5

    F=G/(rho**3)*N
    F[cu.isinf(F)]=0
    U=M*F*rho**2
    U_sum=cu.sum(cu.sum(U,axis=0),axis=0)/2 #poterntial energy update
    F=cu.reshape(F,(3,3,n,1))
    F=F*diff
    F_sum=-F.sum(axis=0)                    #force update   


    #V(h)
    #h=h_n
    
    dt2=h_n/(U_sum)
    dt2=cu.array(np.reshape(dt2,(n,1)))
    t2=t2+dt2
    
    v=v+dt2*F_sum                           #velocity update
    V_norm=0.5*M*((v*v).sum(axis=2))
    T_sum=V_norm.sum(axis=0)                #kinetic energy update 
   
    #X(h/2)
    #h=h_n/2
    
    dt3=0.5*h_n/(B+T_sum)
    dt3=cu.array(np.reshape(dt3,(n,1)))
    t3=t3+dt3
    
    r=r+dt3*v                             #position update

    
    return r,v

In [132]:
T_all2=np.reshape(T_all1,(n,1))
#states=index,instantaneous time,T_orbit,R1x,R2y,R3x,R3y,R3x,R3y,V1x,V2y,V3x,V3y,V3x,V3y
states=np.hstack((np.zeros((n,1)),T_all2,cu.asnumpy(R[0,:]),cu.asnumpy(R[1,:]),cu.asnumpy(R[2,:]),\
                  cu.asnumpy(V[0,:]),cu.asnumpy(V[1,:]),cu.asnumpy(V[2,:])))
df=pd.DataFrame(states)
df.to_csv("output_data.csv",header=False)

In [133]:
T=max(T_all1*10000) #total time of run in days (10000 timesteps of the longest orbit)
print(T," =orbital period of the longest orbit")
w1=0.28
w2=0.62546642846767004501
w3=1-2*(w1+w2)
w=[w1,w2,w3,w2,w1]         #weights to run 4th order LHLA

iter=len(w)
count=0

76822064.29164323  =orbital period of the longest orbit


# Integrator

In [134]:
import time
start=time.time()
while t<T:
    for jk in range(iter):
        h_n=H_n*w[jk]
        r_present,v_present=LHLA2(R,V)
        R,V=r_present,v_present
        t=t2[0]    
    if count%1000==0:
        timer=np.reshape(cu.asnumpy(t2),(n,1))
        states=np.hstack((timer,T_all2,cu.asnumpy(R[0,:]),cu.asnumpy(R[1,:]),cu.asnumpy(R[2,:]),\
                          cu.asnumpy(V[0,:]),cu.asnumpy(V[1,:]),cu.asnumpy(V[2,:])))
        df=pd.DataFrame(states)
        df.to_csv("output_data.csv",mode='a',header=False)
        R1=np.append(R1,R)
    if count%5000==0:                          #step to print data
        print(count,t1[0],t2[0],t3[0])
    count+=1

stop=time.time()
t_elap=stop-start
print(t)
print("The time of execution of above program is :",t_elap/60," mins")

0 [2.66036144] [5.33999971] [2.66077721]
5000 [13142.07550054] [26418.24276593] [13142.5036094]
10000 [26053.50582275] [52371.97983591] [26052.90238605]
15000 [38896.19278553] [78191.982635] [38896.46338098]
20000 [51738.47877706] [104010.6066253] [51738.99007718]
25000 [64583.49083661] [129833.93778008] [64583.44345687]
30000 [77432.45993075] [155665.200345] [77431.81315712]
35000 [90282.95337383] [181500.16756201] [90282.34189646]
40000 [103131.62511949] [207332.051752] [103131.61360176]
45000 [115977.55345371] [233158.331621] [115978.05359857]
50000 [128822.8710372] [258982.76403652] [128823.26810341]
55000 [141670.37720899] [284811.10664739] [141670.17457954]
60000 [154520.69123607] [310645.21162138] [154520.00381273]
65000 [167371.71218737] [336481.33387761] [167371.13576513]
70000 [180220.75759157] [362313.9735545] [180220.77138019]
75000 [193067.18320652] [388141.23878037] [193067.68343856]
80000 [205912.94828691] [413966.5932244] [205913.35859124]
85000 [218760.6628325] [439795

# Data Collection

In [ ]:
columns = ['index','time','T_orbit','R1x','R1y','R2x','R2y','R3x','R3y','V1x','V1y','V2x','V2y','V3x','V3y']
df_input = pd.read_csv("output_data.csv",names=columns )
df_input.head()
N=np.array([50,100,500,1000,5000,10000]) #to store data afte N orbits
idx_list=np.arange(n)
df_final = pd.DataFrame(columns = columns)
for i in idx_list:
    for j,val in enumerate(N):
        df_traverse=df_input[df_input['index']==i]
        diff = np.abs(df_traverse['time'] - val*df_traverse['T_orbit'])
        closest_row_index = diff.idxmin()
        row_to_insert = df_traverse.loc[[closest_row_index]]
        row_to_insert['N']=val
        if df_final.empty:
            df_final = row_to_insert
        else:
            df_final = df_final.append(row_to_insert)

final1 = df_final.reset_index()
final = final1.drop(['level_0'], axis=1)

R_end1=final[['R1x','R1y','R2x','R2y','R3x','R3y','V1x','V1y','V2x','V2y','V3x','V3y']]
R_end=np.array(R_end1)
np.shape(R_end)
R_end[:,1]
R1=R_end[:,0:2]
R2=R_end[:,2:4]
R3=R_end[:,4:6]
V1=R_end[:,6:8]
V2=R_end[:,8:10]
V3=R_end[:,10:12]
R_end=cu.array([R1,R2,R3])
V_end=cu.array([V1,V2,V3])
np.shape(R_end)

# Stability

In [ ]:
#Stability
#conversion from Cartesian to Keplerian elements
r12=R_end[0]-R_end[1]
v12=V_end[0]-V_end[1]
r3=R_end[2]-com
v3=V_end[2]-v_com
mu=G*(M[0]+M[1])
a_b=cu.array([((R_end[0]-R_end[1])*(R_end[0]-R_end[1])).sum(axis=1)**0.5])
a_p=cu.array([(r3*r3).sum(axis=1)**0.5])
e1=(cu.sum(v12*v12,axis=1)*cu.transpose(r12)-cu.sum(v12*r12,axis=1)*cu.transpose(v12))/mu-cu.transpose(r12)/a_b
e1_val=cu.array([((e1)*(e1)).sum(axis=0)**0.5])
e3=(cu.sum(r3*v3,axis=1)*cu.transpose(r3)-cu.sum(r3*r3,axis=1)*cu.transpose(v3))/mu-cu.transpose(r3)/a_p
e3_val=cu.array([((e1)*(e1)).sum(axis=0)**0.5])
st=[]
flag=0
stability=[]
for i in range(n):
    if e1_val[:,i]>1 or e3_val[:,i]>1 or a_b[:,i]>100 or a_b[:,i]<0.001 or a_p[:,i]>1000:
        st.append("unstable")
        stability.append(1)
    else:
        st.append("stable")
        stability.append(0)
for i in st:
    if i=="stable":
        flag+=1
print("no. of stable cases=", flag)
final.to_csv('all_final_results.csv')

# Plot

In [ ]:
import matplotlib.pyplot as plt
siz=np.shape(R1)[0]
R2=cu.asnumpy(R1)

def get_cmap(n, name='hsv'):
    return plt.cm.get_cmap(name, n)  
cmap = get_cmap(46)                 #importing a color map with n colors

#Create figureS
fig=plt.figure(figsize=(10,10))
#Create 3D axes
ax=fig.add_subplot(111)
#Plot the orbits
case=680                              #case number 
step=1
#ax.view_init(90, 90)
for i in range(3):
    ax.plot(R2[0:siz:step,i,case,0],R2[0:siz:step,i,case,1],color=cmap(i*10),label="Body"+str(i+1))
    ax.scatter(R2[0,i,case,0],R2[0,i,case,1],color=cmap(i*10),marker="o",s=100)
    ax.scatter(R2[-1,i,case,0],R2[-1,i,case,1],color=cmap(i*10),marker="x",s=100)
    
ax.set_xlabel("x-coordinate",fontsize=14)
ax.set_ylabel("y-coordinate",fontsize=14)
ax.set_title("Visualization\n",fontsize=14)
ax.legend(loc="upper left",fontsize=14)

plt.show()
    